In [ ]:
from unsloth import FastLanguageModel

In [ ]:
import re
import datetime
import pandas as pd

from src.utils import run

In [ ]:
# Run Training
model, tokenizer = run('stance_detection', do_training=True)

In [ ]:
from src.utils import get_savefile
from src.prompting import get_datasets
from src.metrics import get_metrics
from src.plot import plot_metric

In [ ]:
# from src.aduc import get_data
# from src.aduc import change_lbl
import src.aduc as aduc

# from src.claim_detect import get_data
# from src.claim_detect import change_lbl
import src.claim_detect as cd

# from src.evidence_detect import get_data
# from src.evidence_detect import change_lbl
import src.evidence_detect as ed

# from src.evidence_type import get_data
# from src.evidence_type import change_lbl
import src.evidence_type as et

# from src.fallacies import get_data
# from src.fallacies import change_lbl
import src.fallacies as fd

# from src.quality import get_data
# from src.quality import change_lbl
import src.quality as aq

# from src.relation import get_data
# from src.relation import change_lbl
import src.relation as arc

# from src.stance_detect import get_data
# from src.stance_detect import change_lbl
import src.stance_detect as sd

In [ ]:
checkpoint = {
    'aduc': f'./outputs/aduc/2025-05-07_17:29:07_meta-Llama-3.1-8B-Instruct_2e4000spl2_train_resp/checkpoint-250',
    'claim_detection': f'./outputs/claim_detection/2025-05-09_14:57:11_meta-Llama-3.1-8B-Instruct_2e4000spl2_train_resp/checkpoint-250',
    'evidence_detection': f'./outputs/evidence_detection/2025-05-09_15:51:24_meta-Llama-3.1-8B-Instruct_2e4000spl2_train_resp/checkpoint-250',
    'evidence_type': f'./outputs/evidence_type/2025-05-12_10:47:06_meta-Llama-3.1-8B-Instruct_2e4000spl2_train_resp/checkpoint-250',
    'fallacies': f'./outputs/fallacies/2025-05-09_16:41:44_meta-Llama-3.1-8B-Instruct_2e4000spl2_train_resp/checkpoint-340',
    'relation': f'./outputs/relation/2025-05-09_18:41:38_meta-Llama-3.1-8B-Instruct_2e4000spl2_train_resp/checkpoint-250',
    'stance_detection': f'./outputs/stance_detection/2025-05-15_09:50:35_Meta-Llama-3.1-8B-Instruct_2e4000spl2_train_resp/checkpoint-250',
    'quality': f'./outputs/quality/2025-05-12_09:44:16_meta-Llama-3.1-8B-Instruct_2e4000spl2_train_resp/checkpoint-250',
}
st_model = {
    'aduc': f'./saved_model/aduc',
    'claim_detection': f'./saved_model/claim_detection',
    'evidence_detection': f'./saved_model/evidence_detection',
    'evidence_type': f'./saved_model/evidence_type',
    'fallacies': f'./saved_model/fallacies',
    'relation': f'./saved_model/relation',
    'stance_detection': f'./saved_model/stance_detection',
    'quality': f'./saved_model/quality'
}
d_sys_prt = {
    'aduc': f'You are an expert in argumentation. Your task is to determine whether the given [SENTENCE] is a Claim or a Premise. Utilize the [TOPIC] and the [FULL TEXT] as context to support your decision\nYour answer must be in the following format with only Claim or Premise in the answer section:\n<|ANSWER|><answer><|ANSWER|>.',
    'claim_detection': f'You are an expert in argumentation. Your task is to determiner whether the given [SENTENCE] is a Claim or Non-claim. Utilize the [TOPIC] and the [FULL TEXT] as context to support your decision\nYour answer must be in the following format with only Claim or Non-claim in the answer section:\n<|ANSWER|><answer><|ANSWER|>.',
    'evidence_detection': f'You are an expert in argumentation. Your task is to determine whether the given [SENTENCE] is an Evidence or Non-evidence. Utilize the [TOPIC] and the [ARGUMENT] as context to support your decision\nYour answer must be in the following format with only Evidence or Non-evidence in the answer section:\n<|ANSWER|><answer><|ANSWER|>.',
    'evidence_type': f'You are an expert in argumentation. Your task is to determine the type of evidence of the given [SENTENCE]. The type of evidence would be in the [TYPE] set. Utilize the [TOPIC] and the [CLAIM] as context to support your decision\nYour answer must be in the following format with only the type of evidence in the answer section:\n<|ANSWER|><answer><|ANSWER|>.',
    'fallacies': f'You are an expert in argumentation. Your task is to determine the type of fallacy in the given [SENTENCE]. The fallacy would be in the [FALLACY] Set. Utilize the [TITLE] and the [FULL TEXT] as context to support your decision.\nYour answer must be in the following format with only the fallacy in the answer section:\n<|ANSWER|><answer><|ANSWER|>.',
    'relation': f'You are an expert in argumentation. Your task is to determine the type of relation between [SOURCE] and [TARGET]. The type of relation would be in the [RELATION] set. Utilize the [TOPIC] as context to support your decision\nYour answer must be in the following format with only the type of the relation in the answer section:\n<|ANSWER|><answer><|ANSWER|>.',
    'stance_detection': f'You are an expert in argumentation. Your task is to determine whether the given [SENTENCE] is For or Against. Utilize the [TOPIC] as context to support your decision\nYour answer must be in the following format with only For or Against in the answer section:\n<|ANSWER|><answer><|ANSWER|>.',
    'quality': f'You are an expert in argumentation. Your task is to determine the quality of the [SENTENCE]. The quality would be in the [QUALITY] set. Utilize the [TOPIC], the [STANCE] and the [DEFINITION] as context to support your decision\nYour answer must be in the following format with only the quality in the answer section:\n<|ANSWER|><answer><|ANSWER|>.'
}

In [ ]:
# Load Checkpoint : Parameters\n",
task = 'claim_detection'
task_data = 'claim_detection'
task_title = 'Claim Detection'
ckpt_name = checkpoint.get(task)
st_name = ''
gguf_file = f'./gguf_model/{task}'
max_seq_lenght = 2048
dtype = None
load_in_4bit = True
gpu_mem_use = 0.6
system_prompt = d_sys_prt.get(task_data)
chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{OUTPUT}<|eot_id|>"""
time = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
s_file = get_savefile(
    task_name=task_data,
    spl_name='spl2',
    m_name='Meta-Llama-3.1-8B-Instruct',
    n_sample=4000,
    epoch=2,
    train_resp=f'_train_resp',
    outputs_dir=f'./outputs/test_{task}',
    time=time
)
match task_data:
    case 'aduc':
        labels, tr_d, val_d, test_d = aduc.get_data(s_file)
        change_lbl = aduc.change_lbl
    case 'claim_detection':
        labels, tr_d, val_d, test_d = cd.get_data(s_file)
        change_lbl = cd.change_lbl
    case 'evidence_detection':
        labels, tr_d, val_d, test_d = ed.get_data(s_file)
        change_lbl = ed.change_lbl
    case 'evidence_type':
        labels, tr_d, val_d, test_d = et.get_data(s_file)
        change_lbl = et.change_lbl
    case 'fallacies':
        labels, tr_d, val_d, test_d = fd.get_data(s_file)
        change_lbl = fd.change_lbl
    case 'relation':
        labels, tr_d, val_d, test_d = arc.get_data(s_file)
        change_lbl = arc.change_lbl
    case 'stance_detection':
        labels, tr_d, val_d, test_d = sd.get_data(s_file)
        change_lbl = sd.change_lbl
    case 'quality':
        labels, tr_d, val_d, test_d = aq.get_data(s_file)
        change_lbl = aq.change_lbl
# labels, tr_d, val_d, test_d = get_data(s_file)
print(system_prompt)
print(ckpt_name)
print(gguf_file)
print(change_lbl)


In [ ]:
# Load Checkpoint : Load Model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=ckpt_name,
    max_seq_length=max_seq_lenght,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    fast_inference=True,
    gpu_memory_utilization=gpu_mem_use
)

In [ ]:
# Load Checkpoint : Get Datasets
train_set, val_set, test_set = get_datasets(
    tokenizer=tokenizer,
    train=tr_d,
    val=val_d,
    test=test_d, 
    chat_template=chat_template,
    sys_prt=system_prompt
)

In [ ]:
# Test on fewer data
# from datasets import Dataset
# tmp_set = test_set[:10]
# tmp_set = pd.DataFrame().from_records(tmp_set)
# tmp_set = Dataset.from_pandas(tmp_set)
# tmp_set

In [ ]:
# Load Checkpoint : Run On test_set
from src.training import test
res_test_chpt = test(
    model=model,
    tokenizer=tokenizer,
    data_test=test_set,
    labels=labels,
)
# metric, metric_m = get_metrics(change_lbl, res_test_chpt)
metric, _ = get_metrics(change_lbl, res_test_chpt, is_multi_lbl=False)
plot_metric(
    metric=metric,
    # title=f'{task_title}: Scores ckpt{nbckpt}',
    # file_plot=f'./img/{task}/scores_ckpt{nbckpt}_metric_single.png',
    # file_metric=f'./test_res/{task}/scores_ckpt{nbckpt}_metric_single.csv'
)
# plot_metric(
#     metric=metric_m
# )


In [ ]:
# model.save_pretrained_merged(
#     './saved_model/claim_detection',
#     tokenizer,
#     save_method="merged_16bit",
# )

In [ ]:
model.save_pretrained_merged(
    './saved_model/claim_detection/saved_lora',
    tokenizer,
    save_method="lora",
)

In [ ]:
# Save Method
# model.save_pretrained_gguf(
#     gguf_file,
#     tokenizer,
#     quantization_method = "q8_0", # choose quant method: q8_0
# )

In [ ]:
from ollama import chat

res = []
s = '<[|]ANSWER[|]>'
quant = 'q8_0'
names_dataset = test_d['datasets']
true_labels = test_d['answer']
# print(len(test_d['conversations']))
for i in test_d['conversations']:
    response = chat(model='unsloth_model', messages=[i[1]])
    print(response['message']['content'])
    tmp = re.split(s, response['message']['content'])
    res.append(tmp[1])
d_res = {'names': names_dataset, 'pred': res, 'lbl': true_labels}
df_res = pd.DataFrame(data=d_res)
df_res.to_csv(f'./test_res/{task}/test_result_gguf_{quant}.csv', index=False)

In [ ]:

# metric_single, metric_multi = get_metrics(change_lbl, df_res)
metric_single, _ = get_metrics(change_lbl, df_res, is_multi_lbl=False)
plot_metric(
    metric=metric_single,
    # title=f'{task_title}: Score gguf_{quant} single label',
    # file_plot=f'./img/{task}/scores_gguf_{quant}_metric_single.png',
    # file_metric=f'./test_res/{task}/scores_gguf_{quant}_metric_single.csv'
)
# For Fallacies Task Only
# plot_metric(
#     metric=metric_multi,
#     title=f'{task_title}: Score gguf_{quant} multi label',
#     file_plot=f'./img/{task}/scores_gguf_{quant}_metric_multi.png',
#     file_metric=f'./test_res/{task}/scores_gguf_{quant}_metric_multi.csv'
# )

In [ ]:
# def gen(txt, model, sampling_params):
#     output = model.fast_generate(
#         txt,
#         sampling_params = sampling_params,
#     )[0].outputs[0].text
    
#     return output

# def format_output(answer: str, fallacies: set) -> list:
#     s = '<[|]ANSWER[|]>'
#     tmp = re.split(s, answer)
#     pred= [i for i in tmp if i in fallacies]
#     return pred

# def zero_shot_gen(data: list[str], model, fallacies: set, sampling_params) -> list:
#     res = []
#     for i in data:
#         out = gen(i, model, sampling_params)
#         pred = format_output(out, fallacies)
#         res.append(pred)
#     return res

In [ ]:
# class custom_validation_callback(TrainerCallback):
#     def __init__(self, data, sampling_params, fallacies, n_step=10):
#         super().__init__()
#         self.val_dataset = data
#         self.sampling_params = sampling_params
#         self.fallacies = fallacies
#         self.n_step=n_step
#     def on_step_end(self, args, state, control, **kwargs):
#         if state.global_step % self.n_step == 0 and state.global_step > 0 :
#             model.save_lora('sft_save_lora')
#             FastLanguageModel.for_inference(model)
#             pred = zero_shot_gen(
#                 data=self.val_dataset['text'],
#                 model=model,
#                 fallacies=self.fallacies,
#                 sampling_params=self.sampling_params
#             )
#             tmp_pred = [i if i != [] else ['Failed'] for i in pred]
#             d = pd.DataFrame().from_records(tmp_pred)
#             d['truth_label'] = self.val_dataset['answer']
#             d['step'] = np.full((len(d['truth_label']),), state.global_step)
#             try:
#                 d.to_csv(
#                     './validation_res.csv',
#                     index=False,
#                     mode='a',
#                     header=['pred', 'truth_label', 'step']
#                 )
#             except FileNotFoundError:
#                 d.to_csv('./validation_res.csv', index=False, header=['pred', 'truth_label'])
#         return super().on_step_end(args, state, control, **kwargs)

# class custom_test_callback(TrainerCallback):
#     def __init__(self, data, sampling_params, fallacies):
#         super().__init__()
#         self.test_dataset = data
#         self.sampling_params = sampling_params
#         self.fallacies = fallacies
#     def on_train_end(self, args, state, control, **kwargs):
#         model.save_lora('sft_save_lora')
#         FastLanguageModel.for_inference(model)
#         pred = zero_shot_gen(
#             data=self.test_dataset['text'],
#             model=model,
#             fallacies=self.fallacies,
#             sampling_params=self.sampling_params
#         )
#         tmp_pred = [i if i != [] else ['Failed'] for i in pred]
#         d = pd.DataFrame().from_records(tmp_pred)
#         d['truth_label'] = self.test_dataset['answer']
#         try:
#             d.to_csv('./test_res.csv', index=False, mode='a', header=['pred','truth_label'])
#         except FileNotFoundError:
#             d.to_csv('./test_res.csv', index=False, header=['pred', 'truth_label'])
#         return super().on_train_end(args, state, control, **kwargs)